<a href="https://colab.research.google.com/github/chobocoding/deeplearning/blob/main/data_augmentation%20(ASL%20data)" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os

os.chdir('/content/drive/MyDrive/Colab Notebooks/IT융합/asl_data')

In [ ]:
train_df = pd.read_csv('sign_mnist_train.csv')
valid_df = pd.read_csv('sign_mnist_valid.csv')

In [ ]:
train_df.shape

(27455, 785)

In [ ]:
y_train = train_df['label']
y_valid = valid_df['label']

del train_df['label']
del valid_df['label']

x_train = train_df.values
x_valid = valid_df.values

In [ ]:
from PIL import Image
image = x_train[0, :].reshape(28, 28)
img_pil = Image.fromarray(np.uint8(image))

In [ ]:
img_pil

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
                                transforms.Resize(32),
                                transforms.RandomCrop(28)
])

In [ ]:
transform(img_pil)

1. Data preparation

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda:0")

In [ ]:
class ASL_Dataset(Dataset):
    def __init__(self, path, transform=None):
        super(ASL_Dataset, self).__init__()

        train_df = pd.read_csv(path)
        y_train = train_df['label']
        del train_df['label']
        x_train = train_df.values

        self.x_train = x_train
        self.y_train = y_train
        self.transform = transform

    def __getitem__(self, index):
        x_train_sample = np.uint8(self.x_train[index].reshape(28, 28))

        if self.transform:
            x_train_pil = Image.fromarray(x_train_sample)
            x_train_np = np.array(self.transform(x_train_pil))
            x_train_tensor = torch.tensor(x_train_np[np.newaxis, :, : ]).float().to(device)  # CxHxW >> 1x28x28
        else:
            x_train_tensor = torch.tensor(x_train_sample[np.newaxis, :, : ]).float().to(device)  # CxHxW >> 1x28x28

        y_train_tensor = torch.tensor(self.y_train[index]).to(device)

        return x_train_tensor, y_train_tensor

    def __len__(self):
        return self.x_train.shape[0]


In [ ]:
training_data = ASL_Dataset("sign_mnist_train.csv", transform=transform)
validataion_data = ASL_Dataset("sign_mnist_valid.csv", transform=None)

In [ ]:
train_loader = DataLoader(dataset=training_data, batch_size=1000, shuffle=True)
validation_loader = DataLoader(dataset=validataion_data, batch_size=1000, shuffle=False)

2. Constructing a CNN

In [ ]:
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 4, padding=1)
        self.conv4 = nn.Conv2d(64, 256, 3, padding=1)

        self.relu = nn.ReLU()
        self.norm1 = nn.BatchNorm2d(32)
        self.norm2 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(256*3*3, 256)
        self.fc2 = nn.Linear(256, 24)

    def forward(self, x):
        x = self.conv1(x) # 1*28*28  > 16*28*28
        x = self.relu(x) 
        x = self.pool(x) # 16*14*14

        x = self.conv2(x) # 32*14*14
        x = self.norm1(x)
        x = self.relu(x) 
        x = self.pool(x) # 64*7*7

        x = self.conv3(x) # 64*6*6
        x = self.relu(x)

        x = self.conv4(x) # 256*6*6
        x = self.norm2(x)
        x = self.relu(x) 
        x = self.pool(x) # 256*3*3

        x = x.view(-1, 256*3*3)

        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)

        return x

In [ ]:
convnet = ConvNet().to(device)

Loss function and Optimization

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet.parameters(), lr=0.001)

training of Neural Net

In [ ]:
for epoch in range(6):

    loss_val = 0

    for itr, data in enumerate(train_loader):
        optimizer.zero_grad()

        inputs, labels = data

        pred = convnet(inputs)
        loss = loss_function(pred, labels)

        loss.backward()
        optimizer.step()

        loss_val += loss.item()
    print("Epoch :", epoch+1, ", Loss :", loss_val)

Epoch : 1 , Loss : 49.38955610990524
Epoch : 2 , Loss : 10.613447189331055
Epoch : 3 , Loss : 3.1670643016695976
Epoch : 4 , Loss : 1.2059763707220554
Epoch : 5 , Loss : 0.6057356279343367
Epoch : 6 , Loss : 0.3333871643990278


prediction

In [ ]:
pred_list = []
label_list = []
for itr, data in enumerate(validation_loader):
    inputs, labels = data
    pred_test = convnet(inputs)

    pred_category = torch.argmax(pred_test, dim=1)

    pred_list  = pred_list + list(pred_category.cpu())
    label_list = label_list + list(labels.cpu())

accu = np.mean(np.array(pred_list) == np.array(label_list))
print("Test accuracy :", accu)

Test accuracy : 0.9564974902398216
